# ANALIZA PODATKOV

In [2]:
import pandas as pd

filmi = pd.read_csv("filmi.csv")

In [3]:
filmi["razlika_v_ocenah"] = abs(filmi["ocena_rotten_tomatoes"] / 10 - filmi["ocena_imdb"])

In [4]:
filmi["desetletje"] = (filmi["leto"] // 10) * 10

In [ ]:
#za narediti:
#kje ima film visjo oceno
#katero leto ima koliko filmov